## 第2章 小売店のデータでデータ加工を行う10本ノック

第２章では小売店の売り上げデータの解析、予測になります。汚いデータをいかに加工して予測のモデルを構築をして行くかという演習になっています。

こういう実務的な問題を演習として用意してくれているので、とてもありがたいです。

結果だけ載せました。正解かどうかは保障しません笑

### github
- jupyter notebook形式のファイルは[こちら](https://github.com/hiroshi0530/wa-src/blob/master/ml/data100/02/02_nb.ipynb)

### google colaboratory
- google colaboratory で実行する場合は[こちら](https://colab.research.google.com/github/hiroshi0530/wa-src/blob/master/ml/data100/02/02_nb.ipynb)

### 筆者の環境

In [1]:
!sw_vers

ProductName:	Mac OS X
ProductVersion:	10.14.6
BuildVersion:	18G6020


In [2]:
!python -V

Python 3.7.3


基本的なライブラリをインポートしそのバージョンを確認しておきます。

In [3]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import matplotlib
import matplotlib.pyplot as plt
import scipy
import numpy as np
import pandas as pd

print('matplotlib version :', matplotlib.__version__)
print('scipy version :', scipy.__version__)
print('numpy version :', np.__version__)
print('pandas version :', pd.__version__)

matplotlib version : 3.0.3
scipy version : 1.4.1
numpy version : 1.16.2
pandas version : 1.0.3


## 解答

### ノック11 : データを読み込んでみよう

In [4]:
uriage_data = pd.read_csv('uriage.csv')
uriage_data.head()

,purchase_date,item_name,item_price,customer_name
0,2019-06-13 18:02:34,商品A,100.0,深井菜々美
1,2019-07-13 13:05:29,商 品 S,NaN,浅田賢二
2,2019-05-11 19:42:07,商 品 a,NaN,南部慶二
3,2019-02-12 23:40:45,商品Z,2600.0,麻生莉緒
4,2019-04-22 03:09:35,商品a,NaN,平田鉄二


途中で半角スペースが入っていたり、大文字、小文字の差異などいい感じに表記が揺れています。一応カラム名を確認しておきます。

In [13]:
uriage_data.columns

Index(['purchase_date', 'item_name', 'item_price', 'customer_name',
       'purchase_month'],
      dtype='object')

In [5]:
kokyaku_data = pd.read_excel('kokyaku_daicho.xlsx')
kokyaku_data.head()

,顧客名,かな,地域,メールアドレス,登録日
0,須賀ひとみ,すが ひとみ,H市,suga_hitomi@example.com,2018/01/04
1,岡田 敏也,おかだ としや,E市,okada_toshiya@example.com,42782
2,芳賀 希,はが のぞみ,A市,haga_nozomi@example.com,2018/01/07
3,荻野 愛,おぎの あい,F市,ogino_ai@example.com,42872
4,栗田 憲一,くりた けんいち,E市,kurita_kenichi@example.com,43127


In [16]:
kokyaku_data.columns

Index(['顧客名', 'かな', '地域', 'メールアドレス', '登録日'], dtype='object')

### ノック12 : データの揺れを見てみよう

In [6]:
uriage_data['item_name'].head()

0      商品A
1    商 品 S
2    商 品 a
3      商品Z
4      商品a
Name: item_name, dtype: object

In [7]:
uriage_data['item_price'].head()

0     100.0
1       NaN
2       NaN
3    2600.0
4       NaN
Name: item_price, dtype: float64

かなりデータが揺れがあるのがわかります。

### ノック13 : データに揺れがあるまま集計してみよう 

日付がobject型なのでdatetime型に変換します。

In [8]:
uriage_data['purchase_date'] = pd.to_datetime(uriage_data['purchase_date'])

In [22]:
uriage_data[['purchase_date']].head()

,purchase_date
0,2019-06-13 18:02:34
1,2019-07-13 13:05:29
2,2019-05-11 19:42:07
3,2019-02-12 23:40:45
4,2019-04-22 03:09:35


daetime型に変換されています。

月ごとの集計値を計算します。

In [10]:
uriage_data['purchase_month'] = uriage_data['purchase_date'].dt.strftime('%Y%m')
res = uriage_data.pivot_table(index='purchase_month', columns='item_name', aggfunc='size', fill_value=0)
res

item_name,商品W,商 品 n,商品E,商品M,商品P,商品S,商品W,商品X,商 品O,商 品Q,...,商品k,商品l,商品o,商品p,商品r,商品s,商品t,商品v,商品x,商品y
purchase_month,,,,,,,,,,,,,,,,,,,,,
201901,0,1,0,0,0,0,0,0,0,0,...,1,1,1,0,0,0,0,0,0,0
201902,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,1,1,0,0
201903,0,0,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
201904,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
201905,0,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
201906,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
201907,0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,2,0,0,0,0,0


In [11]:
res.shape

(7, 99)

商品数が99個になっています。
次に価格の集計についても見てみます。

In [12]:
res = uriage_data.pivot_table(index='purchase_month', columns='item_name', values='item_price', aggfunc='sum', fill_value=0)
res

item_name,商品W,商 品 n,商品E,商品M,商品P,商品S,商品W,商品X,商 品O,商 品Q,...,商品k,商品l,商品o,商品p,商品r,商品s,商品t,商品v,商品x,商品y
purchase_month,,,,,,,,,,,,,,,,,,,,,
201901,0,1400,0,0,0,0,0,0,0,0,...,1100,1200,1500,0,0,0,0,0,0,0
201902,0,0,0,0,0,0,0,2400,0,0,...,0,0,0,0,0,1900,2000,2200,0,0
201903,0,0,500,1300,1600,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
201904,2300,0,0,0,0,0,0,0,0,1700,...,0,0,0,0,0,1900,0,0,0,0
201905,0,0,0,0,0,1900,0,0,0,0,...,0,1200,0,0,0,0,0,0,0,2500
201906,0,0,0,0,0,0,2300,0,0,0,...,0,0,0,1600,0,0,0,0,2400,0
201907,0,0,0,0,0,0,0,0,0,0,...,0,0,1500,0,1800,0,0,0,0,0


こちらも全く意味をなしてないことがわかります。

### ノック14 : 

まずは商品名の揺れを補正していくようです。今回抽出された商品の一覧です。

In [24]:
pd.unique(uriage_data['item_name'])

array(['商品A', '商 品 S', '商 品 a', '商品Z', '商品a', '商品S', '商品 a', '商品V', '商品O',
       '商 品U', '商品L', '商  品V', '商 品O', '商品C', '商品I', '商品r', '商品X', '商品 g',
       '商品R', '商品P', '商品Q', '商品y', '商品 A', '商品N', '商品W', '商 品E', '商品K',
       '商品B', '商品F', '商 品s', '  商品W', ' 商 品 n', '商 品F', '商品D', '商品M',
       '商品Y', '商品U', '商品H', '商品T', '商品J', '商  品O', '商品E', '商  品Q', ' 商品S',
       ' 商品M', '商  品T', '商品G', '商 品G', ' 商品P', ' 商品E', '商 品N', '商 品Y',
       '商品 J', '商品 V', '商品 K', '商 品V', '商 品D', '商 品A', '商品 F', '商品 H',
       '商 品K', '商 品T', '商品 X', '商品 Q', '商 品X', '商 品H', '商 品C', '商品 B',
       '商品 O', '商品 T', '商品v', '商品p', '商品i', '商品 w', '商 品 s', '商 品 q',
       '商品s', '商品l', '商品t', '商品k', '商品g', '商品o', '商品 R', '商品 S', '商 品M',
       '商品j', '商品d', '商品 I', '商品 E', '商品 o', '商品c', '商品 v', '商品e', '商品x',
       '商 品I', ' 商品W', ' 商品X', '商品 M', '商 品P'], dtype=object)

商品数は９９個です。

In [25]:
len(pd.unique(uriage_data['item_name']))

99

スペースの有無、半角全角統一をします。文字列を扱うメソッド`str`を利用します。

In [32]:
uriage_data['item_name'] = uriage_data['item_name'].str.upper()
uriage_data['item_name'] = uriage_data['item_name'].str.replace("　","")
uriage_data['item_name'] = uriage_data['item_name'].str.replace(" ","")
uriage_data.sort_values(by=['item_name'], ascending=True).head(3)

,purchase_date,item_name,item_price,customer_name,purchase_month
0,2019-06-13 18:02:34,商品A,100.0,深井菜々美,201906
1748,2019-05-19 20:22:22,商品A,100.0,松川綾女,201905
223,2019-06-25 08:13:20,商品A,100.0,板橋隆,201906


In [30]:
uriage_data['item_name'].unique()

array(['商品A', '商品S', '商品Z', '商品V', '商品O', '商品U', '商品L', '商品C', '商品I',
       '商品R', '商品X', '商品G', '商品P', '商品Q', '商品Y', '商品N', '商品W', '商品E',
       '商品K', '商品B', '商品F', '商品D', '商品M', '商品H', '商品T', '商品J'],
      dtype=object)

In [31]:
len(uriage_data['item_name'].unique())

26

となり、商品名の揺れは解消されました。

### ノック15 :  金額欠損値の補完をしよう

金額にどれぐらいの欠損値（Null)があるか確認しています。

In [36]:
uriage_data.isnull().sum()

purchase_date       0
item_name           0
item_price        387
customer_name       0
purchase_month      0
dtype: int64

In [37]:
uriage_data.shape

(2999, 5)

行数が2999で、387行が欠損値で約１２％が欠落していることになります。

教科書では、

In [38]:
uriage_data.isnull().any(axis=0)

purchase_date     False
item_name         False
item_price         True
customer_name     False
purchase_month    False
dtype: bool

とデータの欠損の有無を確認しています。

In [40]:
uriage_data.isnull().head()

,purchase_date,item_name,item_price,customer_name,purchase_month
0,False,False,False,False,False
1,False,False,True,False,False
2,False,False,True,False,False
3,False,False,False,False,False
4,False,False,True,False,False


anyメソッド、引数のオプションにaxisを指定することで、一つでもNullがあればTrueを返し、Nullの有無を確認出来ます。

In [41]:
uriage_data.isnull().any(axis=0)

purchase_date     False
item_name         False
item_price         True
customer_name     False
purchase_month    False
dtype: bool

In [45]:
flg_is_null = uriage_data['item_price'].isnull()

array(['商品S', '商品A', '商品P', '商品N', '商品W', '商品R', '商品I', '商品L', '商品F',
       '商品O', '商品B', '商品C', '商品V', '商品Q', '商品U', '商品K', '商品T', '商品X',
       '商品E', '商品M', '商品G', '商品J', '商品D', '商品H', '商品Y'], dtype=object)

In [46]:
uriage_data.loc[flg_is_null, 'item_name'].unique()

array(['商品S', '商品A', '商品P', '商品N', '商品W', '商品R', '商品I', '商品L', '商品F',
       '商品O', '商品B', '商品C', '商品V', '商品Q', '商品U', '商品K', '商品T', '商品X',
       '商品E', '商品M', '商品G', '商品J', '商品D', '商品H', '商品Y'], dtype=object)

In [48]:
flg_is_null = uriage_data['item_price'].isnull()

for trg in list(uriage_data.loc[flg_is_null, 'item_name'].unique()):
  price = uriage_data.loc[(~flg_is_null) & (uriage_data['item_name'] == trg), 'item_price'].max()
  uriage_data['item_price'].loc[(flg_is_null) & (uriage_data['item_name'] == trg)] = price
  
uriage_data.head()

/Users/hiroshi/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,purchase_date,item_name,item_price,customer_name,purchase_month
0,2019-06-13 18:02:34,商品A,100.0,深井菜々美,201906
1,2019-07-13 13:05:29,商品S,1900.0,浅田賢二,201907
2,2019-05-11 19:42:07,商品A,100.0,南部慶二,201905
3,2019-02-12 23:40:45,商品Z,2600.0,麻生莉緒,201902
4,2019-04-22 03:09:35,商品A,100.0,平田鉄二,201904


In [51]:
uriage_data['item_price'].isnull().any(axis=0)
uriage_data.isnull().any(axis=0)

purchase_date     False
item_name         False
item_price        False
customer_name     False
purchase_month    False
dtype: bool

### ノック16 : 顧客名の揺れを補正しよう

次は文字列の揺れの修正になります。

In [52]:
kokyaku_data['顧客名'].head()

0     須賀ひとみ
1    岡田　 敏也
2      芳賀 希
3     荻野  愛
4     栗田 憲一
Name: 顧客名, dtype: object

こちらもスペースが有無に違いがある事尾がわかります。

In [53]:
uriage_data['customer_name'].head()

0    深井菜々美
1     浅田賢二
2     南部慶二
3     麻生莉緒
4     平田鉄二
Name: customer_name, dtype: object

こちらはスペースがありません。こういった名前の表式の差異は、名前が一意キーとなっている場合ｌ、ジョインできないなどの大きいな問題になります。

こちらは、スペースを削除する方向でデータの補正を行います。

In [54]:
kokyaku_data['顧客名'] = kokyaku_data['顧客名'].str.replace(" ","")
kokyaku_data['顧客名'] = kokyaku_data['顧客名'].str.replace("　","")
kokyaku_data['顧客名'].head()

0    須賀ひとみ
1     岡田敏也
2      芳賀希
3      荻野愛
4     栗田憲一
Name: 顧客名, dtype: object

事務レベルでは名前の補正は本当にやっかいです。このほかにも、同姓同名の処理や、漢字が同じなのに読み方が異なる場合など、大変です。

### ノック17 : 日付の揺れを補正しよう

次に日付の揺れを補正します。教科書の場合は、日付の表記が異なっていたり、エクセルの設定により日付が数字になっていたりするので、それを補正します。

まずは数値として取り込まれているデータを取得します。

In [58]:
flg_is_series = kokyaku_data['登録日'].astype('str').str.isdigit()

flg_is_series.sum()

22

### ノック18 : 顧客名をキーに二つのデータを結合しよう

### ノック19 : クレンジングしたデータをダンプしよう

### ノック20 : データを集計しよう

## 関連記事
- [第1章 ウェブからの注文数を分析する10本ノック](/ml/data100/01/)
- [第2章 小売店のデータでデータ加工を行う10本ノック](/ml/data100/02/)
- [第3章 顧客の全体像を把握する10本ノック](/ml/data100/03/)
- [第4章 顧客の行動を予測する10本ノック](/ml/data100/04/)
- [第5章 顧客の退会を予測する10本ノック](/ml/data100/05/)
- [第6章 物流の最適ルートをコンサルティングする10本ノック](/ml/data100/06/)
- [第7章 ロジスティクスネットワークの最適設計を行う10本ノック](/ml/data100/07/)
- [第8章 数値シミュレーションで消費者行動を予測する10本ノック](/ml/data100/08/)
- [第9章 潜在顧客を把握するための画像認識10本ノック](/ml/data100/09/)
- [第10章 アンケート分析を行うための自然言語処理10本ノック](/ml/data100/10/)